In [17]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
from nltk.tokenize import RegexpTokenizer

In [2]:
df_all = pd.read_excel(r'D:\TEST Codingan\Sistem-Temu-Kembali-Informasi\Riset Jupiter Notebook\Data\df_final.xlsx').head(1000)

# EDA

In [3]:
df_all = df_all.drop(columns = ['Unnamed: 0'])

In [4]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   divisions          917 non-null    object
 1   title              1000 non-null   object
 2   abstract           758 non-null    object
 3   subjects           1000 non-null   object
 4   publication        498 non-null    object
 5   publisher          863 non-null    object
 6   creators           999 non-null    object
 7   date               1000 non-null   object
 8   keywords           181 non-null    object
 9   uri                1000 non-null   object
 10  new_abstract       758 non-null    object
 11  text_raw           1000 non-null   object
 12  language           1000 non-null   object
 13  language_accuracy  1000 non-null   object
 14  fulltext           1000 non-null   object
 15  fulltext_stopword  1000 non-null   object
 16  text_stemmed       1000 non-null   object
d

In [5]:
pd.DataFrame(df_all[['title', 'abstract']])

,title,abstract
0,TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,Metil metsulfuron merupakan senyawa aktif yang...
1,Media Pembelajaran Digital sebagai Sumber Bela...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...
2,Strategi Keberlanjutan Jasa Air Bersih Berbasi...,Persoalan penyediaan jasa air bersih merupakan...
3,Metode Sexing Spermatozoa XY dan Aplikasi Inse...,NaN
4,PENGARUH BAHAN PELLETING TERHADAP PERKECAMBAHA...,Bawang merah merupakan komoditas pertanian pen...
...,...,...
995,Analisis Kelayakan Finansial Usaha Burung Puyu...,Unggas penghasil telur terbesar kedua setelah ...
996,Buku Teknologi Pengelolaan Agen Biologis Hara,NaN
997,Teknologi Pengelolaan Agen Biologis Hara (TPABH),NaN
998,PENGARUH KECERDASAN EMOSIONAL DAN KEMAMPUAN AW...,Pembelajaran online menyisakan banyak permasal...


In [ ]:
topik_riset = {'phew'}

for i in range(len(df_all['fokus_riset'])):
  topik_riset.add(df_all['fokus_riset'].iloc[i])

In [ ]:
topik_riset

{'Integrasi Fokus Riset Energi-Energi Baru dan Terbarukan',
 'Kebencanaan',
 'Kemaritiman',
 'Kesehatan-Obat',
 'Material Maju',
 'Pangan-Pertanian',
 'Pertahanan dan Keamanan',
 'Sosial Humaniora, Seni Budaya, Pendidikan',
 'Teknologi Informasi dan Komunikasi',
 'Transportasi',
 nan,
 'phew'}

In [ ]:
temporal = []

for i in range(len(df_all['fokus_riset'])):
  if df_all['fokus_riset'].iloc[i] == 'Kesehatan-Obat':
    temporal.append(df_all.iloc[i])

In [ ]:
temporal = []

for i in range(len(df_all['fokus_riset'])):
  if pd.isnull(df_all['fokus_riset'].iloc[i]):
    temporal.append(df_all.iloc[i])

In [ ]:
len(temporal)

1240

In [ ]:
df_temp = pd.DataFrame(temporal)
df_temp[['title', 'fokus_riset']]

In [ ]:
df_all['fokus_riset'].iloc[3921] = 'Kesehatan-Obat'
df_all['fokus_riset'].iloc[3926] = 'Sosial Humaniora, Seni Budaya, Pendidikan'

# Mesin Pencari

In [7]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   divisions          917 non-null    object
 1   title              1000 non-null   object
 2   abstract           758 non-null    object
 3   subjects           1000 non-null   object
 4   publication        498 non-null    object
 5   publisher          863 non-null    object
 6   creators           999 non-null    object
 7   date               1000 non-null   object
 8   keywords           181 non-null    object
 9   uri                1000 non-null   object
 10  new_abstract       758 non-null    object
 11  text_raw           1000 non-null   object
 12  language           1000 non-null   object
 13  language_accuracy  1000 non-null   object
 14  fulltext           1000 non-null   object
 15  fulltext_stopword  1000 non-null   object
 16  text_stemmed       1000 non-null   object
d

In [8]:
df_all['title'].iloc[5]

'PENGENALAN BUNGA KRISAN POTONG DI DESA SUNGAI \nLANGKA KAB. PESAWARAN SEBAGAI KOMODITAS BARU DI \nDESA AGROWISATA'

In [9]:
# Inisialisasi TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

In [10]:
# Contoh teks
text1 = 'Tingkat METIL METSULFURON dalam COpedoda dhipa sp hahahaha'
text2 = 'Tingkat METIL METSULFURON dalam TOKSISITAS MASKULINITAS COPEPODA Daphnia sp.'

# Inisialisasi TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transformasi teks menjadi vektor TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform([text1, text2])

# Hitung similaritas kosinus antara kedua teks
cosine_similarities = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

# Tampilkan hasil similaritas
print("Similaritas antara teks 1 dan teks 2:", cosine_similarities[0][0])

Similaritas antara teks 1 dan teks 2: 0.42113589132819584


In [11]:
dicari = 'intro kembang KRISAN POTONG DI DESA pesawaran'
dicari = dicari.lower()

In [12]:
def remove_stopwords(text):
    factory = StopWordRemoverFactory()
    stopwords = factory.get_stop_words()

    result = []
    for x in  text.split():
        if x not in stopwords:
            result.append(x)

    hasil_stopword = " ".join(result)

    return hasil_stopword

In [67]:
def stem(text):
    factory2 = StemmerFactory()
    stemmer = factory2.create_stemmer()
    return stemmer.stem(text)

In [13]:
dicari = remove_stopwords(dicari)
dicari

'intro kembang krisan potong desa pesawaran'

In [28]:
from tqdm import tqdm

df_temu = []
similaritas = []

for i in tqdm(range(len(df_all['text_stemmed']))):
  temp = df_all['text_stemmed'].iloc[i]

  # Transformasi teks menjadi vektor TF-IDF
  tfidf_matrix = tfidf_vectorizer.fit_transform([dicari, temp])

  # Hitung similaritas kosinus antara kedua teks
  cosine_similarities = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

  if cosine_similarities[0][0] > 0.1:
    df_temu.append(df_all.iloc[i])
    similaritas.append(cosine_similarities)

df_temu = pd.DataFrame(df_temu)
df_temu['similaritas'] = similaritas

100%|██████████| 1000/1000 [00:04<00:00, 246.29it/s]


In [29]:
len(df_temu)

20

In [30]:
df_temu[['title', 'similaritas','abstract']].sort_values(by=['similaritas'], ascending=False)

,title,similaritas,abstract
5,PENGENALAN BUNGA KRISAN POTONG DI DESA SUNGAI ...,[[0.29700048400793727]],Tujuan kegiatan ini adalah mengenalkan \nkomod...
364,MODEL REGULASI REVITALISASI BUMDES UNTUK MENIN...,[[0.20420476491133877]],Badan Usaha Milik Desa (BUMDes) bukan hal baru...
809,PENDAMPINGAN PEMETAAN POTENSI DESA DONO ARUM B...,[[0.2003808302844586]],Pembangunan desa menjadi salah satu fokus utam...
976,BANTUAN TEKNIS PEMBUATAN PETA DESA SUKOHARJO 3...,[[0.1711863996883662]],"Dalam penyelenggaraan pemerintahan yang baik, ..."
706,LAPORAN AKHIR PENGABDIAN DIPA FT UNIVERSITAS L...,[[0.16243361005558013]],Dalam penyelenggaraan pemerintahan yang ba...
133,Manfaat program Smart Village di Desa Hanura K...,[[0.15830078629952526]],Saat ini pembangunan wilayah pedesaan mengalam...
66,Pengembangan Pertanian Frontier di Desa Sidoka...,[[0.1480350210986175]],Gisting merupakan representasi wilayah pertani...
246,PEMAMPUDAYAAN KELOMPOK SADAR WISATAMELALUI PEN...,[[0.14764566818120411]],Tujuan kegiatan pengabdian kepada masyarak...
128,Pembangunan Dan Pemberdayaan Masyarakat Desa M...,[[0.14362799865308862]],NaN
129,Pendirian Badan Usaha Milik Desa (Bumdes) Untu...,[[0.12442854619985262]],NaN


In [32]:
tokenizer = RegexpTokenizer(r'\w+')

topik = []

for index in range(len(df_temu)):
  # Ambil dataset contoh (dalam kasus ini, dataset 20 newsgroups)
  teks = df_temu['fulltext_stopword'].iloc[index]
  data = tokenizer.tokenize(teks)

  # Proses vektorisasi teks dengan TF-IDF
  tfidf_vectorizer = TfidfVectorizer(max_features=5000)
  tfidf_matrix = tfidf_vectorizer.fit_transform(data)

  # Terapkan LSA dengan TruncatedSVD
  num_topics = 4  # Jumlah topik yang ingin diidentifikasi
  lsa_model = TruncatedSVD(n_components=num_topics, random_state=42)
  lsa_topic_matrix = lsa_model.fit_transform(tfidf_matrix)

  # Ambil kata-kata kunci untuk setiap topik
  terms = tfidf_vectorizer.get_feature_names_out()
  top_keywords = 1  # Jumlah kata kunci untuk setiap topik (HANYA BISA DIAMBIL 1)

  kata_kunci = []

  for i, topic in enumerate(lsa_model.components_):
      top_keywords_idx = topic.argsort()[-top_keywords:][::-1]
      keywords = [terms[key_idx] for key_idx in top_keywords_idx]
      kata_kunci.append(keywords[0])

  topik.append(kata_kunci)

df_temu['topik'] = topik

In [33]:
df_temu[['title', 'similaritas','abstract','keywords','topik']].sort_values(by=['similaritas'], ascending=False)

,title,similaritas,abstract,keywords,topik
5,PENGENALAN BUNGA KRISAN POTONG DI DESA SUNGAI ...,[[0.29700048400793727]],Tujuan kegiatan ini adalah mengenalkan \nkomod...,NaN,"[potong, komoditas, krisan, bunga]"
364,MODEL REGULASI REVITALISASI BUMDES UNTUK MENIN...,[[0.20420476491133877]],Badan Usaha Milik Desa (BUMDes) bukan hal baru...,NaN,"[bumdes, desa, esa, ndang]"
809,PENDAMPINGAN PEMETAAN POTENSI DESA DONO ARUM B...,[[0.2003808302844586]],Pembangunan desa menjadi salah satu fokus utam...,NaN,"[desa, potensi, kegiatan, masyarakat]"
976,BANTUAN TEKNIS PEMBUATAN PETA DESA SUKOHARJO 3...,[[0.1711863996883662]],"Dalam penyelenggaraan pemerintahan yang baik, ...","Peta Desa, Batas Desa, Titik Batas","[desa, peta, batas, titik]"
706,LAPORAN AKHIR PENGABDIAN DIPA FT UNIVERSITAS L...,[[0.16243361005558013]],Dalam penyelenggaraan pemerintahan yang ba...,NaN,"[desa, batas, praktik, peta]"
133,Manfaat program Smart Village di Desa Hanura K...,[[0.15830078629952526]],Saat ini pembangunan wilayah pedesaan mengalam...,NaN,"[desa, program, smart, smart]"
66,Pengembangan Pertanian Frontier di Desa Sidoka...,[[0.1480350210986175]],Gisting merupakan representasi wilayah pertani...,NaN,"[pertanian, pengembangan, desa, frontier]"
246,PEMAMPUDAYAAN KELOMPOK SADAR WISATAMELALUI PEN...,[[0.14764566818120411]],Tujuan kegiatan pengabdian kepada masyarak...,"pendidikan dan pelatihan, community capacity, ...","[desa, wisata, kunjir, pengembangan]"
128,Pembangunan Dan Pemberdayaan Masyarakat Desa M...,[[0.14362799865308862]],NaN,NaN,"[desa, pembangunan, masyarakat, pemberdayaan]"
129,Pendirian Badan Usaha Milik Desa (Bumdes) Untu...,[[0.12442854619985262]],NaN,NaN,"[desa, mewujudkan, masyarakat, ekonomi]"


# Test SCRIPT STKI

In [78]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm

def search(teks):
  df = pd.read_excel(r'D:\TEST Codingan\Sistem-Temu-Kembali-Informasi\Riset Jupiter Notebook\Data\df_final.xlsx').head(1000)
  df = df.drop(columns='Unnamed: 0')

  teks = stem(teks)

  df_temu = []
  similaritas = []

  # Inisialisasi TF-IDF Vectorizer
  tfidf_vectorizer = TfidfVectorizer()

  for i in tqdm(range(len(df['text_stemmed']))):
    temp = df['text_stemmed'].iloc[i]

    # Transformasi teks menjadi vektor TF-IDF
    tfidf_matrix = tfidf_vectorizer.fit_transform([teks, temp])

    # Hitung similaritas kosinus antara kedua teks
    cosine_similarities = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

    if cosine_similarities[0][0] > 0.1:
      df_temu.append(df.iloc[i])
      similaritas.append(cosine_similarities)

  df_temu = pd.DataFrame(df_temu)
  df_temu['similaritas'] = similaritas

  return latent_semantic_analysis(df_temu)

def remove_stopwords(text):
  factory = StopWordRemoverFactory()
  stopwords = factory.get_stop_words()

  result = []
  for x in  text.split():
      if x not in stopwords:
          result.append(x)

  hasil_stopword = " ".join(result)

  return hasil_stopword

def stem(text):
  factory2 = StemmerFactory()
  stemmer = factory2.create_stemmer()
  return stemmer.stem(text)

def latent_semantic_analysis(dataset):
  tokenizer = RegexpTokenizer(r'\w+')
  topik = []

  for index in range(len(dataset)):
    # Ambil dataset
    teks = dataset['fulltext_stopword'].iloc[index]
    data = tokenizer.tokenize(teks)

    # Proses vektorisasi teks dengan TF-IDF
    tfidf_vectorizer = TfidfVectorizer(max_features=5000)
    tfidf_matrix = tfidf_vectorizer.fit_transform(data)

    # Terapkan LSA dengan TruncatedSVD
    num_topics = 4  # Jumlah topik yang ingin diidentifikasi
    lsa_model = TruncatedSVD(n_components=num_topics, random_state=42)
    lsa_topic_matrix = lsa_model.fit_transform(tfidf_matrix)

    # Ambil kata-kata kunci untuk setiap topik
    terms = tfidf_vectorizer.get_feature_names_out()
    top_keywords = 1  # Jumlah kata kunci untuk setiap topik (HANYA BISA DIAMBIL 1)

    kata_kunci = []

    for i, topic in enumerate(lsa_model.components_):
        top_keywords_idx = topic.argsort()[-top_keywords:][::-1]
        keywords = [terms[key_idx] for key_idx in top_keywords_idx]
        kata_kunci.append(keywords[0])

    topik.append(kata_kunci)
  dataset['topik'] = topik
  # return dataset
  return dataset.iloc[0].to_dict()
    
dicari = 'intro kembang KRISAN POTONG DI DESA pesawaran'
# dicari = input('Masukan Pencarian: ')

dataset = search(dicari)

100%|██████████| 1000/1000 [00:03<00:00, 253.79it/s]


In [81]:
dataset['title']

'PENGENALAN BUNGA KRISAN POTONG DI DESA SUNGAI \nLANGKA KAB. PESAWARAN SEBAGAI KOMODITAS BARU DI \nDESA AGROWISATA'